In [ ]:
import math
import random
from Crypto.Util.number import *

# Given values from the challenge
e = 65537
n = 3191868707489083296976422171754481125088448532695639929013026951283334085716937496519972309690132954050242378974370025245594553866043111294840209514577676946872746793700126873931085112786381515154186105142460622301297252278473097650013016482539838576476763183025029834004241446095147665598581368214114851984460699747964946764645986828307675081596907634022110868102739948513844625534865764252668312850364286204872187001344218083941399088833989233474318289529103178632284291007694811574023047207470113594082533713524606268388742119103653587354956091145288566437795469230667897089543048576812362251576281067933183713438502813206542834734983616378764909202774603304124497453696792428111112644362307853143219890039129054302905340695668256116233515323529918746264727874817221051242387145263342018617858562987223211598238069486447049955021864781104312134816578626968386395835285074116149472750100154961405785440009296096563521430833
phi = 3191868707489083296976422171754481125088448532695639929013026951283334085716937496519972309690132954050242378974370025245594553866043111294840209514577676946872746793700126873931085112786381515154186105142460622301297252278473097650013016482539838576476763183025029834004241446095147665598581368214114851984394758254181484105857103844940487787404078873566779953101987404891507588290232992132681729619718279684673827347612899406697514777723904351697638562060304399923174376216080338949397741477013367831377040866937433520175862575061413321076151761545984886547872427147498175814451096795344136954743868643889768901204954902708679102384061694877757565486240670882343628571424084461972849147495569088820011108794930593172573959423278140327579049114196086428504291102619820322231225943837444001821535593671764186251713714593498207219093585758479440828038119079608764008747539277397742897542501803218788455452391287578171880267200
c = 8847973599594272436100870059187158819529199340583461915617467299706215012295598155778224026186157290320191983062022702191439286020733625396165573681688842631368993650799220713225485752608650482408353598320160571916055498330875851476520668973214124194890108144336715482373743731578734960096351460142579903010557821654345995923836938260379746304222820835040419844947019844885128550552066290798665884099701340641403329066058638137944934073185448687990744852400616823426082588916251127609191094346267837812018236673478691437630461425526779014305216914035039981685211625653600564431704400207095883904994772993227506462664

def gcd(a, b):
    """Calculate the greatest common divisor of a and b."""
    while b:
        a, b = b, a % b
    return a

def pollard_rho(n):
    """Pollard's Rho algorithm for factorization"""
    if n % 2 == 0:
        return 2
    
    def f(x):
        return (x * x + 1) % n
    
    x = random.randint(1, n-1)
    y = x
    d = 1
    
    while d == 1:
        x = f(x)
        y = f(f(y))
        d = gcd(abs(x - y), n)
        
        if d == n:
            # Start over with a new random value
            return pollard_rho(n)
    
    return d

def factor_n_with_d(n, e, d):
    """
    Uses the private key d to factorize the RSA modulus n
    """
    # Calculate k*φ(n) = e*d - 1
    k_phi_n = e * d - 1
    
    # Factor k*φ(n) = 2^s * t, where t is odd
    s = 0
    t = k_phi_n
    while t % 2 == 0:
        s += 1
        t //= 2
    
    # Try with different random bases
    for _ in range(100):  # Try multiple times with different random bases
        a = random.randint(2, n-1)
        
        # Check if a and n have common factors
        gcd_a_n = math.gcd(a, n)
        if gcd_a_n > 1:
            return gcd_a_n
        
        # Compute a^t mod n
        x = pow(a, t, n)
        if x == 1 or x == n-1:
            # This base doesn't work, try another one
            continue
        
        # Try to find a suitable i
        for i in range(1, s):
            x_prev = x
            x = pow(x, 2, n)
            
            if x == 1:
                # Found a factor: gcd(x_prev - 1, n)
                factor = math.gcd(x_prev - 1, n)
                if 1 < factor < n:
                    return factor
            
            if x == n-1:
                # This base won't work, try another one
                break
        
        # If we get here and x != n-1, we've found a suitable base
        if x != n-1:
            factor = math.gcd(x - 1, n)
            if 1 < factor < n:
                return factor
    
    # If we try many times and don't find a factor, use Pollard's Rho
    return pollard_rho(n)

# Start the factorization process
print("Starting RSA challenge solution...")

# 1. Compute d = e^(-1) mod phi
d = pow(e, -1, phi)
print(f"Computed d = {d}")

# 2. Use factor_n_with_d to find one factor of n
p = factor_n_with_d(n, e, d)
print(f"Found first factor p = {p}")

# 3. Compute qr = n/p
qr = n // p
print(f"Computing q*r = n/p = {qr}")

# 4. Use Pollard's Rho to find q or r
try:
    q = pollard_rho(qr)
    r = qr // q
    # Make sure q and r are different
    if q == r:
        r = pollard_rho(r)
        q = qr // r
    
    print(f"Found q = {q}")
    print(f"Found r = {r}")
    
    # Verify the factorization
    assert p*q*r == n, "Factorization is incorrect!"
    print("Factorization verified: p*q*r = n")
    
    # 5. Compute n2 = p*q (the actual encryption modulus)
    n2 = p * q
    print(f"Computing n2 = p*q = {n2}")
    
    # 6. Compute phi_2 = (p-1)*(q-1)
    phi_2 = (p - 1) * (q - 1)
    print(f"Computing phi_2 = (p-1)*(q-1) = {phi_2}")
    
    # 7. Compute d2 = e^(-1) mod phi_2
    d2 = pow(e, -1, phi_2)
    print(f"Computing d2 = e^(-1) mod phi_2 = {d2}")
    
    # 8. Decrypt the message
    m = pow(c, d2, n2)
    flag = long_to_bytes(m)
    print(f"Decrypted flag: {flag.decode()}")
    
except Exception as e:
    print(f"Error during factorization: {e}")

Starting RSA challenge solution...
Computed d = 983319791937448948928909831815966155233467749135983797957993410537719382800936387914281107353766335692424029654890074457307384264696437387167917818931646208361090037151617583726270784856753326990295824691796694451427308596706773907163324576689188410501943156770608241886959042932405074514173029345911147314721915393623818058459418750161717021341964881781486599220732192574569147314948870274178618505914400519303058946065649007754746530391162684602214360864817520399909831939250530896872612423835389116308382365739456837700087971535009764751628912913521138584334716329828771071208136538108215589915295297620190642161344576127809192931232824504965519434322583351610813141539165132478322539755184703347361403277074761984135194481834469890501869197790850740673232484884785271004904890460014867888014459560750704493982023858930509540465683453067731362712507808067213106132609335347367580166670909669153896510120696647745399737473
Found first factor p = 2100

In [ ]:
from Crypto.Util.number import *

# Given values from the challenge
e = 65537
n = 3191868707489083296976422171754481125088448532695639929013026951283334085716937496519972309690132954050242378974370025245594553866043111294840209514577676946872746793700126873931085112786381515154186105142460622301297252278473097650013016482539838576476763183025029834004241446095147665598581368214114851984460699747964946764645986828307675081596907634022110868102739948513844625534865764252668312850364286204872187001344218083941399088833989233474318289529103178632284291007694811574023047207470113594082533713524606268388742119103653587354956091145288566437795469230667897089543048576812362251576281067933183713438502813206542834734983616378764909202774603304124497453696792428111112644362307853143219890039129054302905340695668256116233515323529918746264727874817221051242387145263342018617858562987223211598238069486447049955021864781104312134816578626968386395835285074116149472750100154961405785440009296096563521430833
phi = 3191868707489083296976422171754481125088448532695639929013026951283334085716937496519972309690132954050242378974370025245594553866043111294840209514577676946872746793700126873931085112786381515154186105142460622301297252278473097650013016482539838576476763183025029834004241446095147665598581368214114851984394758254181484105857103844940487787404078873566779953101987404891507588290232992132681729619718279684673827347612899406697514777723904351697638562060304399923174376216080338949397741477013367831377040866937433520175862575061413321076151761545984886547872427147498175814451096795344136954743868643889768901204954902708679102384061694877757565486240670882343628571424084461972849147495569088820011108794930593172573959423278140327579049114196086428504291102619820322231225943837444001821535593671764186251713714593498207219093585758479440828038119079608764008747539277397742897542501803218788455452391287578171880267200
c = 8847973599594272436100870059187158819529199340583461915617467299706215012295598155778224026186157290320191983062022702191439286020733625396165573681688842631368993650799220713225485752608650482408353598320160571916055498330875851476520668973214124194890108144336715482373743731578734960096351460142579903010557821654345995923836938260379746304222820835040419844947019844885128550552066290798665884099701340641403329066058638137944934073185448687990744852400616823426082588916251127609191094346267837812018236673478691437630461425526779014305216914035039981685211625653600564431704400207095883904994772993227506462664

# Remember we know from the original code:
# n = p*q*r
# phi = (p-1)*(q-1)*(r-1)
# c = pow(bytes_to_long(m), e, p*q)

print("Starting direct RSA challenge solution...")

# If we expand phi = (p-1)*(q-1)*(r-1), we get:
# phi = p*q*r - p*q - p*r - q*r + p + q + r - 1
# phi = n - (p*q + p*r + q*r) + (p + q + r) - 1

# Let's calculate n - phi
n_minus_phi = n - phi
print(f"n - phi = {n_minus_phi}")

# n - phi = p*q + p*r + q*r - p - q - r + 1

# Now, from the structure of the challenge, we know:
# 1. The ciphertext c was encrypted with modulus p*q
# 2. We need to find p*q to decrypt

# Looking at n - phi, we have:
# n - phi = (p*q) + p*r + q*r - p - q - r + 1

# From the challenge description, p, q, r are 1024-bit primes
# This means p*q, p*r, q*r are all roughly the same size (much larger than p, q, r)
# So, n - phi ≈ p*q + p*r + q*r

# Let's try a direct approach - let's find a number d that satisfies:
# 1. c^d mod n = m
# 2. (or approximately) c^d mod (p*q) = m

# First, calculate d1 = e^(-1) mod phi(n)
d1 = pow(e, -1, phi)
print(f"d1 (private exponent for n) = {d1}")

# Try to decrypt using this d1
try:
    m1 = pow(c, d1, n)
    flag1 = long_to_bytes(m1)
    print(f"Attempt 1 - Decryption with d1 and n: {flag1}")
except:
    print("Decryption with d1 and n failed")

# Looking at the challenge again:
# If c = m^e mod (p*q)
# Then m = c^d mod (p*q), where d = e^(-1) mod ((p-1)*(q-1))

# We don't know (p-1)*(q-1) directly, but we do know (p-1)*(q-1)*(r-1) = phi
# Let's try a different approach based on what we know about factoring n with d:

# We can try to compute some values that might help us find p*q:
# phi(n) = (p-1)*(q-1)*(r-1)
# phi(n) + n = p*q*r + (p-1)*(q-1)*(r-1)
# phi(n) + n = n + n - p*q - p*r - q*r + p + q + r - 1
# phi(n) + n = 2*n - (p*q + p*r + q*r) + (p + q + r) - 1

# From the original code, we know that the target is p*q
# Let's try another approach: let's try a direct search for the value p*q

# Let's try RSA with the assumption that n2 = p*q
# The challenge is to find n2 and phi2 = (p-1)*(q-1)

# A key observation: phi = (p-1)*(q-1)*(r-1)
# So, (p-1)*(q-1) = phi / (r-1)

# Unfortunately, we don't know r directly.

# Let's try a different approach:
# If n = p*q*r and phi = (p-1)*(q-1)*(r-1),
# By modular arithmetic, p ≡ 0 (mod p), so n ≡ 0 (mod p)
# Similarly, phi + 1 ≡ 1 (mod p)
# So gcd(n, phi+1) might give us a factor

import math

# Try to find a factor using gcd(n, phi+1)
potential_factor = math.gcd(n, phi+1)
print(f"GCD(n, phi+1) = {potential_factor}")

if potential_factor > 1 and potential_factor < n:
    # We found a factor! Let's call it p
    p = potential_factor
    qr = n // p
    print(f"Found factor p = {p}")
    print(f"qr = n/p = {qr}")
    
    # Now, let's find a factor of qr
    # Using the relation phi = (p-1)*(q-1)*(r-1)
    phi_qr = phi // (p-1)
    
    # Try gcd(qr, phi_qr+1)
    potential_q = math.gcd(qr, phi_qr+1)
    
    if potential_q > 1 and potential_q < qr:
        q = potential_q
        r = qr // q
        print(f"Found factor q = {q}")
        print(f"Found factor r = {r}")
        
        # Verify factorization
        if p*q*r == n:
            print("Factorization verified: p*q*r = n")
            
            # Calculate n2 = p*q
            n2 = p * q
            print(f"Calculated n2 = p*q = {n2}")
            
            # Calculate phi2 = (p-1)*(q-1)
            phi2 = (p-1) * (q-1)
            print(f"Calculated phi2 = (p-1)*(q-1) = {phi2}")
            
            # Calculate d2 = e^(-1) mod phi2
            d2 = pow(e, -1, phi2)
            print(f"Calculated d2 = e^(-1) mod phi2 = {d2}")
            
            # Decrypt the message
            m = pow(c, d2, n2)
            flag = long_to_bytes(m)
            print(f"Decrypted flag: {flag.decode()}")
        else:
            print("Error: Factorization is incorrect!")
    else:
        print("Could not find a factor of qr")
else:
    print("Could not find a factor of n")

# Try alternative factorization methods
print("\nTrying alternative factorization methods...")

# Since we were given phi(n), we can try to solve:
# phi(n) = n - (p + q + r) + (pq + pr + qr) - pqr
# phi(n) = n - (p + q + r) + (pq + pr + qr) - n
# phi(n) = (pq + pr + qr) - (p + q + r)

# n - phi(n) = (p + q + r) - (pq + pr + qr) + n
# n - phi(n) = (p + q + r) - (pq + pr + qr) + pqr
# n - phi(n) = (p + q + r) - (pq + pr + qr - pqr)
# n - phi(n) = (p + q + r) - ((p)(q)(r))((1/r) + (1/q) + (1/p) - 1/(pqr))
# n - phi(n) = (p + q + r) - pqr((1/r) + (1/q) + (1/p) - 1/(pqr))
# n - phi(n) = (p + q + r) - pqr((1/r) + (1/q) + (1/p) - 1/(pqr))
# n - phi(n) = (p + q + r) - (pq + pr + qr - 1)

# From the context of the problem, we know c is encrypted with modulus p*q
# Let's approach from another angle and try direct decryption with various moduli

# Try approach with trial moduli derived from the context
from Crypto.Util.number import long_to_bytes

n_diff = n - phi
print(f"n - phi = {n_diff}")

# Just for trial, let's try decrypting with n_diff as the modulus
try:
    m_diff = pow(c, d1, n_diff)
    flag_diff = long_to_bytes(m_diff)
    print(f"Attempting decryption with n_diff as modulus: {flag_diff}")
except:
    print("Decryption with n_diff failed")

# As a last resort, try a brute force approach by modifying the initial e*d relation
print("\nTrying brute force approximation...")

# We know e*d ≡ 1 mod phi(n)
# Let's compute e*d - 1
ed_minus_1 = e * d1 - 1
print(f"e*d - 1 = {ed_minus_1}")

# Check if e*d - 1 is divisible by phi(n)
if ed_minus_1 % phi == 0:
    print("e*d - 1 is divisible by phi(n), as expected")
    k = ed_minus_1 // phi
    print(f"k = {k}")
else:
    print("Error: e*d - 1 is not divisible by phi(n)!")

# Try GCD-based prime separation
def find_factor(n, e, d):
    # Calculate k*φ(n) = e*d - 1
    k_phi_n = e * d - 1
    
    # Factor k*φ(n) = 2^s * t, where t is odd
    s = 0
    t = k_phi_n
    while t % 2 == 0:
        s += 1
        t //= 2
    
    import random
    for _ in range(30):  # Try a few random bases
        a = random.randint(2, n-1)
        
        # Check if a and n have common factors
        gcd_a_n = math.gcd(a, n)
        if 1 < gcd_a_n < n:
            return gcd_a_n
        
        # Compute a^t mod n
        x = pow(a, t, n)
        if x == 1 or x == n-1:
            continue
        
        for i in range(1, s):
            x_prev = x
            x = pow(x, 2, n)
            
            if x == 1:
                factor = math.gcd(x_prev - 1, n)
                if 1 < factor < n:
                    return factor
            
            if x == n-1:
                break
        
        if x != n-1:
            factor = math.gcd(x - 1, n)
            if 1 < factor < n:
                return factor
    
    return None

factor = find_factor(n, e, d1)
if factor:
    print(f"Found factor using the RSA attack: {factor}")
    
    # Check if this is p
    qr = n // factor
    
    # Generate d for p*q and try to decrypt
    for trial_q in [factor, qr]:
        for trial_p in [factor, qr]:
            if trial_p == trial_q:
                continue
            
            trial_n2 = trial_p * trial_q
            trial_phi2 = (trial_p - 1) * (trial_q - 1)
            
            try:
                trial_d2 = pow(e, -1, trial_phi2)
                trial_m = pow(c, trial_d2, trial_n2)
                try:
                    trial_flag = long_to_bytes(trial_m)
                    if b'NSSCTF{' in trial_flag:
                        print(f"Success! Decrypted flag: {trial_flag.decode()}")
                        print(f"Using p = {trial_p}, q = {trial_q}")
                        exit(0)  # Success! Exit the script
                except:
                    pass
            except:
                pass
else:
    print("Factor-finding attack was unsuccessful")

print("All methods failed to decrypt the message.")